In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
from math import pi
from builtins import range
import numpy as np

In [2]:
def Loss_ASoftmax(x, y, num_cls, l = 0, m = 4, name = 'asoftmax',reuse =None):
    '''
    x: B x D - data
    y: B x 1 - label
    l: 1 - lambda 
    '''
    xs = x.get_shape()
    with tf.variable_scope("logita",reuse=reuse):
        
        w = tf.get_variable("asoftmax", [xs[1], num_cls], dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer())

    eps = 1e-8

    xw = tf.matmul(x,w) 
    print("logits_origin shape = ",xw.get_shape().as_list())
    if m == 0:
        return xw, tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=xw))

    w_norm = tf.norm(w, axis = 0) + eps
    print("w_norm shape = ",w_norm.get_shape().as_list())
    
    logits = xw/w_norm
    print("logits shape = ",logits.get_shape().as_list())

    if y is None: 
        return logits, None

    ordinal = tf.constant(list(range(0, xs[0])), tf.int32)
    ordinal_y = tf.stack([ordinal, y], axis = 1)

    x_norm = tf.norm(x, axis = 1) + eps

    sel_logits = tf.gather_nd(logits, ordinal_y)
    print("logits_inputs shape = ",sel_logits.get_shape().as_list())

    cos_th = tf.div(sel_logits, x_norm)

    if m == 1:

        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits))

    else:

        if m == 2:

            cos_sign = tf.sign(cos_th)
            res = 2*tf.multiply(tf.sign(cos_th), tf.square(cos_th)) - 1

        elif m == 4:

            cos_th2 = tf.square(cos_th)
            cos_th4 = tf.pow(cos_th, 4)
            sign0 = tf.sign(cos_th)
            sign3 = tf.multiply(tf.sign(2*cos_th2 - 1), sign0)
            sign4 = 2*sign0 + sign3 - 3
            res = sign3*(8*cos_th4 - 8*cos_th2 + 1) + sign4
        else:
            raise ValueError('unsupported value of m')

        scaled_logits = tf.multiply(res, x_norm)

        f = 1.0/(1.0+l)
        ff = 1.0 - f
        comb_logits_diff = tf.add(logits, tf.scatter_nd(ordinal_y, tf.subtract(scaled_logits, sel_logits), logits.get_shape())) 
        updated_logits = ff*logits + f*comb_logits_diff

        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=updated_logits))

    return logits, loss


In [3]:
np.random.seed(100)
batch = 64
feating = 128
classess = 10000
# inputsinputs =  [[-1.0,1.0],[1.0,1.0],[1.0,1.0]]

inputsinputs = np.random.rand(batch,feating)  
print(inputsinputs)
inputslables = np.random.randint(0,classess,batch)
print(inputslables)

[[ 0.54340494  0.27836939  0.42451759 ...,  0.49216697  0.40288033
   0.3542983 ]
 [ 0.50061432  0.44517663  0.09043279 ...,  0.91975536  0.84960733
   0.25446654]
 [ 0.87755554  0.43513019  0.72949434 ...,  0.34675413  0.1095646   0.378327  ]
 ..., 
 [ 0.875816    0.86369521  0.79803885 ...,  0.57726726  0.6909711
   0.98480009]
 [ 0.00651008  0.0863602   0.13983792 ...,  0.46388836  0.85936029
   0.15810968]
 [ 0.71404555  0.19379255  0.78947814 ...,  0.52351113  0.88957582
   0.5727949 ]]
[9016  890  914 1780 2288 9813 7903 6490 5664 2976 8969 7790 8353 6827 1413
 2096 4131  579 6857 9526 7748 6914 6954 9818 5342 8427 5066 7299 5295 5693
 1840   24 2012 6806  697 8271 6594 4345 7388 9502 7829 4865  811 4477 4915
 1388 6582 2586 7080 1061 5339 8807 3704 7610 1772 6572 5861 9173 8936 6054
 8614 5882 6673 3000]


In [4]:
inputs_place = tf.placeholder(dtype=tf.float32,shape=(batch,feating),name='inputs')
labels_place = tf.placeholder(dtype=tf.int32,shape=(batch),name='labels')

In [13]:
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.001

In [6]:
_,loss = Loss_ASoftmax(inputs_place,labels_place,classess)

logits_origin shape =  [64, 10000]
w_norm shape =  [10000]
logits shape =  [64, 10000]
logits_inputs shape =  [64]


In [26]:
learning_rate = tf.train.exponential_decay(learning_rate=starter_learning_rate,global_step=global_step,
                                           decay_steps=500,decay_rate=0.9,staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

In [27]:
train_op = optimizer.minimize(loss,global_step=global_step)

In [25]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for eopch in range(5000):    
        print(sess.run([loss,train_op,global_step,learning_rate],feed_dict={inputs_place:inputsinputs,labels_place:inputslables}))

[28.837284, None, 1, 0.001]
[27.95154, None, 2, 0.001]
[26.976276, None, 3, 0.001]
[25.843861, None, 4, 0.001]
[24.563095, None, 5, 0.001]
[23.191364, None, 6, 0.001]
[21.794733, None, 7, 0.001]
[20.440834, None, 8, 0.001]
[19.182156, None, 9, 0.001]
[18.054588, None, 10, 0.001]
[17.081402, None, 11, 0.001]
[16.27655, None, 12, 0.001]
[15.645076, None, 13, 0.001]
[15.180486, None, 14, 0.001]
[14.863273, None, 15, 0.001]
[14.66164, None, 16, 0.001]
[14.538671, None, 17, 0.001]
[14.458413, None, 18, 0.001]
[14.38842, None, 19, 0.001]
[14.304697, None, 20, 0.001]
[14.19238, None, 21, 0.001]
[14.051331, None, 22, 0.001]
[13.882406, None, 23, 0.001]
[13.685004, None, 24, 0.001]
[13.456988, None, 25, 0.001]
[13.194447, None, 26, 0.001]
[12.89236, None, 27, 0.001]
[12.545244, None, 28, 0.001]
[12.147906, None, 29, 0.001]
[11.696274, None, 30, 0.001]
[11.188322, None, 31, 0.001]
[10.624979, None, 32, 0.001]
[10.01086, None, 33, 0.001]
[9.3547573, None, 34, 0.001]
[8.669672, None, 35, 0.001]
[7

[2.5756085, None, 279, 0.001]
[2.575598, None, 280, 0.001]
[2.5755868, None, 281, 0.001]
[2.575577, None, 282, 0.001]
[2.575567, None, 283, 0.001]
[2.5755556, None, 284, 0.001]
[2.575546, None, 285, 0.001]
[2.5755336, None, 286, 0.001]
[2.5755217, None, 287, 0.001]
[2.57551, None, 288, 0.001]
[2.5755005, None, 289, 0.001]
[2.5754881, None, 290, 0.001]
[2.5754771, None, 291, 0.001]
[2.5754659, None, 292, 0.001]
[2.5754566, None, 293, 0.001]
[2.5754454, None, 294, 0.001]
[2.5754349, None, 295, 0.001]
[2.5754237, None, 296, 0.001]
[2.5754132, None, 297, 0.001]
[2.5754008, None, 298, 0.001]
[2.5753903, None, 299, 0.001]
[2.5753818, None, 300, 0.001]
[2.5753758, None, 301, 0.001]
[2.5753655, None, 302, 0.001]
[2.575357, None, 303, 0.001]
[2.575345, None, 304, 0.001]
[2.5753345, None, 305, 0.001]
[2.575326, None, 306, 0.001]
[2.5753164, None, 307, 0.001]
[2.5753069, None, 308, 0.001]
[2.5752954, None, 309, 0.001]
[2.5752888, None, 310, 0.001]
[2.5752797, None, 311, 0.001]
[2.5752683, None, 3

[2.5738082, None, 544, 0.00090000004]
[2.5738039, None, 545, 0.00090000004]
[2.5737991, None, 546, 0.00090000004]
[2.5737953, None, 547, 0.00090000004]
[2.5737934, None, 548, 0.00090000004]
[2.573792, None, 549, 0.00090000004]
[2.5737882, None, 550, 0.00090000004]
[2.5737841, None, 551, 0.00090000004]
[2.5737815, None, 552, 0.00090000004]
[2.5737777, None, 553, 0.00090000004]
[2.5737748, None, 554, 0.00090000004]
[2.5737712, None, 555, 0.00090000004]
[2.5737681, None, 556, 0.00090000004]
[2.5737629, None, 557, 0.00090000004]
[2.5737607, None, 558, 0.00090000004]
[2.5737591, None, 559, 0.00090000004]
[2.573756, None, 560, 0.00090000004]
[2.5737543, None, 561, 0.00090000004]
[2.57375, None, 562, 0.00090000004]
[2.5737457, None, 563, 0.00090000004]
[2.5737422, None, 564, 0.00090000004]
[2.57374, None, 565, 0.00090000004]
[2.5737352, None, 566, 0.00090000004]
[2.5737324, None, 567, 0.00090000004]
[2.5737281, None, 568, 0.00090000004]
[2.5737238, None, 569, 0.00090000004]
[2.5737224, None, 

[2.5732679, None, 761, 0.00090000004]
[2.5732651, None, 762, 0.00090000004]
[2.5732636, None, 763, 0.00090000004]
[2.5732627, None, 764, 0.00090000004]
[2.5732601, None, 765, 0.00090000004]
[2.5732598, None, 766, 0.00090000004]
[2.5732601, None, 767, 0.00090000004]
[2.5732589, None, 768, 0.00090000004]
[2.5732574, None, 769, 0.00090000004]
[2.5732548, None, 770, 0.00090000004]
[2.5732539, None, 771, 0.00090000004]
[2.5732522, None, 772, 0.00090000004]
[2.5732493, None, 773, 0.00090000004]
[2.5732489, None, 774, 0.00090000004]
[2.5732467, None, 775, 0.00090000004]
[2.5732427, None, 776, 0.00090000004]
[2.5732419, None, 777, 0.00090000004]
[2.5732396, None, 778, 0.00090000004]
[2.5732398, None, 779, 0.00090000004]
[2.5732377, None, 780, 0.00090000004]
[2.5732362, None, 781, 0.00090000004]
[2.5732365, None, 782, 0.00090000004]
[2.5732341, None, 783, 0.00090000004]
[2.5732341, None, 784, 0.00090000004]
[2.5732324, None, 785, 0.00090000004]
[2.5732315, None, 786, 0.00090000004]
[2.5732288, 

[2.5730176, None, 980, 0.00090000004]
[2.5730181, None, 981, 0.00090000004]
[2.5730174, None, 982, 0.00090000004]
[2.5730155, None, 983, 0.00090000004]
[2.5730147, None, 984, 0.00090000004]
[2.573014, None, 985, 0.00090000004]
[2.5730128, None, 986, 0.00090000004]
[2.5730114, None, 987, 0.00090000004]
[2.5730116, None, 988, 0.00090000004]
[2.5730109, None, 989, 0.00090000004]
[2.5730095, None, 990, 0.00090000004]
[2.573009, None, 991, 0.00090000004]
[2.573009, None, 992, 0.00090000004]
[2.5730062, None, 993, 0.00090000004]
[2.5730054, None, 994, 0.00090000004]
[2.5730071, None, 995, 0.00090000004]
[2.5730081, None, 996, 0.00090000004]
[2.5730088, None, 997, 0.00090000004]
[2.5730069, None, 998, 0.00090000004]
[2.5730062, None, 999, 0.00090000004]
[2.5730052, None, 1000, 0.00090000004]
[2.5730035, None, 1001, 0.00081]
[2.5730033, None, 1002, 0.00081]
[2.5730045, None, 1003, 0.00081]
[2.5730023, None, 1004, 0.00081]
[2.5730023, None, 1005, 0.00081]
[2.5730019, None, 1006, 0.00081]
[2.572

...

[2.5728211, None, 4022, 0.00043046716]
[2.5728192, None, 4023, 0.00043046716]
[2.5728202, None, 4024, 0.00043046716]
[2.5728207, None, 4025, 0.00043046716]
[2.5728226, None, 4026, 0.00043046716]
[2.5728216, None, 4027, 0.00043046716]
[2.5728197, None, 4028, 0.00043046716]
[2.5728197, None, 4029, 0.00043046716]
[2.572819, None, 4030, 0.00043046716]
[2.5728216, None, 4031, 0.00043046716]
[2.5728211, None, 4032, 0.00043046716]
[2.5728223, None, 4033, 0.00043046716]
[2.5728216, None, 4034, 0.00043046716]
[2.5728199, None, 4035, 0.00043046716]
[2.5728188, None, 4036, 0.00043046716]
[2.5728199, None, 4037, 0.00043046716]
[2.5728197, None, 4038, 0.00043046716]
[2.5728207, None, 4039, 0.00043046716]
[2.572823, None, 4040, 0.00043046716]
[2.572823, None, 4041, 0.00043046716]
[2.5728235, None, 4042, 0.00043046716]
[2.5728233, None, 4043, 0.00043046716]
[2.5728202, None, 4044, 0.00043046716]
[2.5728197, None, 4045, 0.00043046716]
[2.5728207, None, 4046, 0.00043046716]
[2.5728192, None, 4047, 0.00

[2.5728197, None, 4235, 0.00043046716]
[2.5728202, None, 4236, 0.00043046716]
[2.5728216, None, 4237, 0.00043046716]
[2.572823, None, 4238, 0.00043046716]
[2.5728219, None, 4239, 0.00043046716]
[2.5728221, None, 4240, 0.00043046716]
[2.5728219, None, 4241, 0.00043046716]
[2.5728216, None, 4242, 0.00043046716]
[2.5728226, None, 4243, 0.00043046716]
[2.5728192, None, 4244, 0.00043046716]
[2.5728197, None, 4245, 0.00043046716]
[2.5728188, None, 4246, 0.00043046716]
[2.5728178, None, 4247, 0.00043046716]
[2.5728211, None, 4248, 0.00043046716]
[2.5728221, None, 4249, 0.00043046716]
[2.5728211, None, 4250, 0.00043046716]
[2.5728219, None, 4251, 0.00043046716]
[2.5728221, None, 4252, 0.00043046716]
[2.5728223, None, 4253, 0.00043046716]
[2.5728204, None, 4254, 0.00043046716]
[2.5728207, None, 4255, 0.00043046716]
[2.5728216, None, 4256, 0.00043046716]
[2.5728197, None, 4257, 0.00043046716]
[2.5728188, None, 4258, 0.00043046716]
[2.5728183, None, 4259, 0.00043046716]
[2.5728188, None, 4260, 0.

[2.5728207, None, 4447, 0.00043046716]
[2.5728214, None, 4448, 0.00043046716]
[2.5728209, None, 4449, 0.00043046716]
[2.5728223, None, 4450, 0.00043046716]
[2.5728207, None, 4451, 0.00043046716]
[2.5728204, None, 4452, 0.00043046716]
[2.5728211, None, 4453, 0.00043046716]
[2.5728233, None, 4454, 0.00043046716]
[2.5728221, None, 4455, 0.00043046716]
[2.5728216, None, 4456, 0.00043046716]
[2.5728188, None, 4457, 0.00043046716]
[2.5728192, None, 4458, 0.00043046716]
[2.5728226, None, 4459, 0.00043046716]
[2.5728211, None, 4460, 0.00043046716]
[2.5728226, None, 4461, 0.00043046716]
[2.5728216, None, 4462, 0.00043046716]
[2.5728207, None, 4463, 0.00043046716]
[2.572823, None, 4464, 0.00043046716]
[2.5728233, None, 4465, 0.00043046716]
[2.5728209, None, 4466, 0.00043046716]
[2.5728199, None, 4467, 0.00043046716]
[2.5728216, None, 4468, 0.00043046716]
[2.5728207, None, 4469, 0.00043046716]
[2.5728188, None, 4470, 0.00043046716]
[2.5728202, None, 4471, 0.00043046716]
[2.5728211, None, 4472, 0.

[2.5728211, None, 4659, 0.00038742041]
[2.5728209, None, 4660, 0.00038742041]
[2.5728197, None, 4661, 0.00038742041]
[2.5728216, None, 4662, 0.00038742041]
[2.572818, None, 4663, 0.00038742041]
[2.5728178, None, 4664, 0.00038742041]
[2.5728202, None, 4665, 0.00038742041]
[2.5728202, None, 4666, 0.00038742041]
[2.5728219, None, 4667, 0.00038742041]
[2.5728219, None, 4668, 0.00038742041]
[2.5728221, None, 4669, 0.00038742041]
[2.5728199, None, 4670, 0.00038742041]
[2.5728199, None, 4671, 0.00038742041]
[2.5728207, None, 4672, 0.00038742041]
[2.5728228, None, 4673, 0.00038742041]
[2.5728219, None, 4674, 0.00038742041]
[2.572823, None, 4675, 0.00038742041]
[2.5728223, None, 4676, 0.00038742041]
[2.5728211, None, 4677, 0.00038742041]
[2.5728207, None, 4678, 0.00038742041]
[2.5728197, None, 4679, 0.00038742041]
[2.5728197, None, 4680, 0.00038742041]
[2.5728192, None, 4681, 0.00038742041]
[2.5728197, None, 4682, 0.00038742041]
[2.5728199, None, 4683, 0.00038742041]
[2.5728202, None, 4684, 0.0

[2.5728202, None, 4872, 0.00038742041]
[2.5728173, None, 4873, 0.00038742041]
[2.5728202, None, 4874, 0.00038742041]
[2.5728202, None, 4875, 0.00038742041]
[2.5728202, None, 4876, 0.00038742041]
[2.5728211, None, 4877, 0.00038742041]
[2.5728209, None, 4878, 0.00038742041]
[2.5728221, None, 4879, 0.00038742041]
[2.5728226, None, 4880, 0.00038742041]
[2.5728221, None, 4881, 0.00038742041]
[2.5728223, None, 4882, 0.00038742041]
[2.5728219, None, 4883, 0.00038742041]
[2.5728216, None, 4884, 0.00038742041]
[2.5728209, None, 4885, 0.00038742041]
[2.5728202, None, 4886, 0.00038742041]
[2.5728211, None, 4887, 0.00038742041]
[2.5728207, None, 4888, 0.00038742041]
[2.5728216, None, 4889, 0.00038742041]
[2.5728211, None, 4890, 0.00038742041]
[2.5728204, None, 4891, 0.00038742041]
[2.5728209, None, 4892, 0.00038742041]
[2.5728211, None, 4893, 0.00038742041]
[2.5728226, None, 4894, 0.00038742041]
[2.5728226, None, 4895, 0.00038742041]
[2.572824, None, 4896, 0.00038742041]
[2.5728211, None, 4897, 0.